### Importamos las librerías necesarias

In [237]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Iniciamos con la lectura de los datos

In [238]:
df = pd.read_excel("real_estate_data.xlsx", index_col = [0])

#### Formateo de los nombres de las columnas para remover espacios

In [239]:
def format_cols_spaces(columns):
  return {key:key.replace(" ","_") for key in columns}

### Funcion para estandarizar la data

In [240]:

def standardize_data(data):
    new_data = data.copy()
    for col in new_data.columns:
        new_data[col] = (new_data[col] - new_data[col].mean()) / new_data[col].std()
    return new_data

### Formateamos la data y normalizamos los datos de train y test

In [249]:
columns_replace = format_cols_spaces(df.columns)
df = df.rename(columns = columns_replace)
df_train = df.loc[0:315].copy()
df_test = df.loc[316:].copy()
mean_std = df_train.describe().loc[["mean","std"]]
df_train_norm = standardize_data(df_train)
df_test_norm = df_test.apply(lambda x: (x - mean_std.loc["mean"]) / mean_std.loc["std"], axis = 1)


### Creamos la clase NeuralNetwork

In [215]:
class NeuralNetwork:
  def __init__(self, activation, eps, learning_rate, epochs):

    #Inicializacion de pesos
    self.W1 = np.random.random((5,6))
    self.b1 = np.random.random((5,1))

    self.W2 = np.random.random((1,5))
    self.b2 = np.random.random((1,1))

    #Inicializacion de hiperparametros
    self.eps = eps
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.activation = self.sigmoid if activation == "sigmoid" else self.relu

  def relu(self,X):
    return np.maximum(0,X)
  
  def sigmoid(self,X):
    return 1/(1+np.exp(-X))

  def forward(self, X):
    zi = self.W1@X + self.b1
    zi = self.activation(zi)
    y_hat = self.W2@zi + self.b2
    return y_hat 
  
  def funcion_objetivo(self, X, y):
    return 0.5*np.mean((self.forward(X).T - y)**2)
  
  def numerical_gradient(self, X, y):
      # Calcular el gradiente numérico por cada parámetro
      eps = self.eps
      grads = {}
      for param_name in ["W1", "b1", "W2", "b2"]:
          param = getattr(self, param_name)
          loss_mat = np.zeros_like(param)
          ## For loop para W1
          for i in range(param.shape[0]):
            for j in range(param.shape[1]):
                original_value = param[i, j]
                param[i, j] = original_value + eps
                grad_plus = self.funcion_objetivo(X, y)
                param[i, j] = original_value - eps
                grad_minus = self.funcion_objetivo(X, y)
                param[i, j] = original_value
                loss_mat[i, j] = (grad_plus - grad_minus) / (2 * eps)
          grads[param_name] = loss_mat
      return grads
  
#funcion fit y loop de entrenamiento
  def fit(self, X, y):
    learning_rate = self.learning_rate
    epochs = self.epochs
    # Inicializar historial de pérdidas
    loss_history = []

    # Ciclo de entrenamiento
    for epoch in range(epochs):
        # Calculamos los gradientes
        grads = self.numerical_gradient(X, y)

        # Actualizamos los pesos y bias
        for param_name in ["W1", "b1", "W2", "b2"]:
            param = getattr(self, param_name)
            grad = grads[param_name]
            param -= learning_rate * grad
        loss = self.funcion_objetivo(X, y)
        loss_history.append(loss)
    return loss_history[-1]



#### Separamos ambos datasets en X e y

In [216]:
df_train_X = df_train.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y = df_train[['Y_house_price_of_unit_area']]

df_train_X_norm = df_train_norm.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y_norm = df_train_norm[['Y_house_price_of_unit_area']]

#### Nos guardamos los valores del Dataset sin las columnas y trasponemos para que queden compatibles las dimensiones con los pesos y biases.
- Aclaración:
No normalizamos los y, dado que esos valores queremos mantenerlos.
Lo que normalizamos son los X de train para que ningún feature quede
dominante solo por una diferencia de escala.

In [217]:
train_X_values = df_train_X.values.T
train_y_values = df_train_y.values

train_X_norm_values = df_train_X_norm.values.T



#### Inicializamos el ciclo de entrenamiento.
- Rangos:
- Epsilon (1e-4, 1e-2)
- Learning Rate (1e-4, 1e-2)
- Epochs (10^2,10^5)
- Funciones de activación (ReLu, Sigmoide)

In [219]:
training_list = []
for eps in range(2,5):
    for learning_rate in range(1,5):
        for epochs in range(2,5):
            for activation in ["sigmoid", "relu"]:
                neural1 = NeuralNetwork(activation, 10**(-eps), 10**(-learning_rate), 10**(epochs))
                loss = neural1.fit(train_X_norm_values, train_y_values)
                training_list.append([eps, learning_rate, epochs, activation, loss])


In [224]:
data_loss = pd.DataFrame(training_list, columns = ["eps", "learning_rate", "epochs", "activation", "loss"])

In [230]:
data_loss = data_loss.sort_values(by = "loss", ascending = True)
data_loss['learning_rate'] = data_loss['learning_rate'].apply(lambda x: 10**(-x))
data_loss['eps'] = data_loss['eps'].apply(lambda x: 10**(-x))
data_loss['epochs'] = data_loss['epochs'].apply(lambda x: 10**(x))

In [232]:
data_loss.reset_index(inplace = True, drop = True)

In [233]:
data_loss

,eps,learning_rate,epochs,activation,loss
0,0.0001,0.0100,10000,sigmoid,24.224737
1,0.0100,0.0100,10000,sigmoid,24.676444
2,0.0010,0.0100,10000,sigmoid,24.697756
3,0.0010,0.0100,10000,relu,27.750065
4,0.0100,0.0100,10000,relu,28.053974
5,0.0001,0.0100,10000,relu,28.332409
6,0.0100,0.0100,1000,relu,28.816681
7,0.0100,0.0010,10000,relu,29.101905
8,0.0100,0.0100,1000,sigmoid,29.647417
9,0.0010,0.0100,1000,relu,29.741471


In [110]:
df_test_X = df_test.drop(['Y_house_price_of_unit_area'], axis = 1)
df_test_y = df_test[['Y_house_price_of_unit_area']]

df_test_X_norm = df_test_norm.drop(['Y_house_price_of_unit_area'], axis = 1)
df_test_y_norm = df_test_norm[['Y_house_price_of_unit_area']]

In [151]:
mse = np.mean((preds.T - df_test_y.values)**2)
mse

66.00471078521562